<a href="https://colab.research.google.com/github/Sid-Oya/DS-Unit-2-Applied-Modeling/blob/master/Copy_DS_232_WrangleMLDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

### A note in preparation for Unit 3

When you're doing your initial data exploration, you're educating yourself about the data, assessing data integrity, and formulating a plan of attack for your predictive model.

The best answer to any of these questions may vary from dataset to dataset. *Experiment* with a simple model to help you through the exploratory data analysis phase.

#### Meaningful Variation
  - Are there any features that are simply constant or quasi-constant values? 
  - Duplicated features?
  - Duplicated rows?
  - Are any of your features highly correlated together?
    - Linear models can be particularly sensitive to multi-collinearity.
    - Larger (esp. wide) datasets tend to have redundant features.

#### Categorical Encodings

  - What are your high cardinality categories?
  - Are there any rare labels that might benefit from grouping together?
  - Are there any categories that could be transformed into a meaningful rank (custom ordinal encoding)?

#### Distributions

  - What are the frequencies of your categorical labels?
  - Is your target feature normally distributed? (Assumption for linear regression model)

#### Outliers
  - How sensitive is your model type to outliers?
    - Less sensitive models include tree-based models. 
    - Linear models, neural networks, and other distance-based models will almost always benefit from scaling.
  - What strategy will you use to identify and handle outliers?

#### Feature Selection

  Why should we reduce the number of features?
  - Reduces potential overfitting
  - Fewer features -> easier interpretation for your stakeholders.
  - Easier implementation and maintain by software engineers.
  - Reduced computational resource requirement.

#### Reproducibility

  - Always set a random seed.
  - Comment, comment, comment!
  - Print out versions of your software.
  - Implement version control for your *data* as well as your *code* (esp. with timestamps!)
  - Wrap your code in reproducible functions / classes for modularity of steps, including feature loading, data wrangling, feature processing, etc. (i.e., *use sklearn pipelines!*)
  - Combine your modularized functions / classes in a single, centralized pipeline to execute your code 
  - Print out / record your final model parameters (optimized hyperparameter values).
  - Record other details of the model: final features passed in, transformations employed, etc. (Jupyter makes this very transparent, but long notebooks can be more confusing than long-form scripts.)

# Explore tabular data for supervised machine learning 🍌

Wrangling your dataset is often the most challenging and time-consuming part of the modeling process.

In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

Let’s get set up:

In [ ]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [ ]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [ ]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [ ]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

order_products__train.csv
departments.csv
orders.csv
products.csv
order_products__prior.csv
aisles.csv


In [ ]:
print (glob('*.csv'))

['order_products__train.csv', 'departments.csv', 'orders.csv', 'products.csv', 'order_products__prior.csv', 'aisles.csv']


In [ ]:
# For each csv file, look at its shape & head

import pandas as pd

from IPython.display import display

def preview():
  for filename in glob('*.csv'):
    df = pd.read_csv(filename)
    print ('\n', filename, df.shape)
    display(df.head())

preview()




 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 orders.csv (3421083, 7)


...

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13



 order_products__prior.csv (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



 aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


### The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

### ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

### Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## Follow Along

### What was the most frequently ordered product?

In [ ]:
order_products__train = pd.read_csv('order_products__train.csv')

In [ ]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [ ]:
order_products__train['product_id'].value_counts(normalize = True)

24852    1.352432e-02
13176    1.117999e-02
21137    7.867880e-03
21903    7.066214e-03
47626    5.875271e-03
             ...     
44256    7.222214e-07
2764     7.222214e-07
4815     7.222214e-07
43736    7.222214e-07
46835    7.222214e-07
Name: product_id, Length: 39123, dtype: float64

In [ ]:
# Merging two tables: brings the tables together on one or more common columns
products = pd.read_csv('products.csv')
merged = order_products__train.merge(products)
merged['product_name'].value_counts().sort_values(ascending=False)[:10]

Banana                    18726
Bag of Organic Bananas    15480
Organic Strawberries      10894
Organic Baby Spinach       9784
Large Lemon                8135
Organic Avocado            7409
Organic Hass Avocado       7293
Strawberries               6494
Limes                      6033
Organic Raspberries        5546
Name: product_name, dtype: int64

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [ ]:
order_products__train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB


In [ ]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   order_id           1384617 non-null  int64 
 1   product_id         1384617 non-null  int64 
 2   add_to_cart_order  1384617 non-null  int64 
 3   reordered          1384617 non-null  int64 
 4   product_name       1384617 non-null  object
 5   aisle_id           1384617 non-null  int64 
 6   department_id      1384617 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 84.5+ MB


### How often are bananas included in a customer's next order?

> Indented block



There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include bananas?"

Let's wrangle!

In [ ]:
train = pd.merge(order_products__train, products, how='inner',
                 left_on='product_id', right_on='product_id')

In [ ]:
### How often are bananas included in a customer's next order?

In [ ]:
# Maybe doesn't answer the question, as it doesn't address the 'next order'
sum(train['product_name'].str.contains('Banana'))

42597

In [ ]:
# Banana = train[train['product_name'] == 'Banana'] + train['reordered']
# sum(Banana['reordered'])
sum(train['product_name'].str.contains('Banana') & train['reordered'] == 1)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,ordered_banana
0,1,49302,1,1,Bulgarian Yogurt,120,16,False
1,816049,49302,7,1,Bulgarian Yogurt,120,16,False
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,False
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,False
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,False


In [ ]:
train['ordered_banana'] = train['product_name'] == 'Banana'
train['ordered_banana']
#train['ordered_banana'].value_counts(normalize=True)

0          False
1          False
2          False
3          False
4          False
           ...  
1384612    False
1384613    False
1384614    False
1384615    False
1384616    False
Name: ordered_banana, Length: 1384617, dtype: bool

In [ ]:
another_banana = train[(train['product_name'] == 'Banana') & (train['reordered'] == 1)]
len(another_banana)

16557

In [ ]:
train['bananas'] = train['product_name'] == 'Banana'
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()
train_wrangled['bananas'].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

In [ ]:
train = order_products__train.merge(products)
train = train.merge(pd.read_csv('orders.csv'))
train['ordered_bananas'] = train['product_name'] == 'Bananas'
train.groupby('order_hour_of_day')['ordered_bananas'].count().sort_values(ascending=False)

order_hour_of_day
14    119370
15    116198
13    114762
11    114119
12    111752
10    110479
16    110237
17     96944
9      93856
18     76522
8      67386
19     58175
20     40920
7      36302
21     34813
22     27319
23     16965
6      11847
0       9083
1       5626
5       3847
2       3226
3       2438
4       2431
Name: ordered_bananas, dtype: int64

In [ ]:
orders= pd.read_csv('orders.csv')
banana_hour = pd.merge(train_wrangled,orders, on=['order_id'])
banana_hour['order_hour_of_day'].value_counts()

14    11103
15    11075
13    10711
12    10622
11    10610
10    10585
16    10495
17     9353
9      9033
18     7558
8      6343
19     5721
20     3978
7      3376
21     3050
22     2442
23     1526
6      1121
0       852
1       507
5       402
2       305
3       223
4       218
Name: order_hour_of_day, dtype: int64

In [ ]:
banana_hour['order_id'].nunique()
banana_hour.shape

131209

In [ ]:
banana_hour.shape

(131209, 8)

In [ ]:


# What is the most common hour of the day that bananas are ordered?


In [ ]:
orders = train.merge(pd.read_csv('orders.csv'))
bo = orders[orders['product_name'] == 'Banana']
bo['order_hour_of_day'].value_counts()

14    1563
15    1519
10    1503
11    1476
16    1467
12    1458
13    1453
9     1372
17    1318
18    1143
8     1025
19     824
20     557
7      533
21     457
22     359
23     215
6      165
0      100
1       69
5       53
2       47
4       27
3       23
Name: order_hour_of_day, dtype: int64

In [ ]:
merged_2 = train.merge(orders)
not_banana = merged_2[merged_2['product_name']!='Banana'].index
merged_2.drop(not_banana, inplace=True)
pd.crosstab(merged_2.order_hour_of_day, merged_2.ordered_banana, margins=True)

AttributeError: ignored

# Join relational data for supervised machine learning

## Overview
Often, you’ll need to join data from multiple relational tables before you’re ready to fit your models.

### Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

## Follow Along

### How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought bananas.

(And *none* of the orders from customers who have *never* bought bananas.)

### What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

## Challenge

**Continue to clean and explore your data.** Can you **engineer features** to help predict your target? For the evaluation metric you chose, what score would you get just by guessing? Can you **make a fast, first model** that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint. But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset today. Follow the instructions in the assignment notebook. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!